# Práctica 5: Análisis de Imagen (Fractografía SEM)

## Importación de Librerías

In [1]:
# Importaciones estándar
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import glob
import json
import re

# Importaciones de Scikit-image
from skimage import exposure, filters, measure, morphology, segmentation
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.util import img_as_float, img_as_ubyte
from skimage.feature import canny  # <--- IMPORTACIÓN AÑADIDA/CORREGIDA

# Importaciones de SciPy
from scipy import ndimage as ndi

# Widgets interactivos
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Output
from IPython.display import display, clear_output

# Configuración de Matplotlib para interactividad
%matplotlib widget

## Configuración de Directorios y Carga de Archivos

In [2]:
# --- Configuración ---
# Define el directorio donde están tus imágenes de entrada
# CORRECCIÓN: Subimos tres niveles ('../../../') para llegar
# a la carpeta raíz y luego bajar a 'files/p5'
FILES_DIR = Path('../../../files/p5') 

# Define el directorio donde se guardarán los resultados
RESULTS_DIR = Path('../data/images')
# --- Fin Configuración ---

# Crear el directorio de resultados si no existe
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Buscar todas las imágenes individuales (jpg, png) en el directorio
image_files = []
for ext in ('*.jpg', '*.jpeg', '*.png'):
    # Usamos glob para encontrar los archivos
    image_files.extend(glob.glob(str(FILES_DIR / ext)))

# Crear una lista de opciones para el Dropdown
# (solo el nombre del archivo, no la ruta completa)
image_options = [Path(f).name for f in image_files]

# Mapear el nombre del archivo (opción) a su ruta completa
image_path_map = {Path(f).name: f for f in image_files}

print(f"Directorio de imágenes: {FILES_DIR.resolve()}")
print(f"Directorio de resultados: {RESULTS_DIR.resolve()}")
if not image_options:
    print(f"ADVERTENCIA: No se encontraron imágenes en '{FILES_DIR.resolve()}'.")
    print("Asegúrate de que tus archivos .jpg o .png estén en ese directorio.")
else:
    print(f"Imágenes encontradas: {image_options}")

Directorio de imágenes: C:\Users\edgar\Documents\GitHub\Aero-Messure\files\p5
Directorio de resultados: C:\Users\edgar\Documents\GitHub\Aero-Messure\Practicas\P05_Analisis_Imagen\data\images
Imágenes encontradas: ['fatigue3.png', 'fatigue4.png', 'fatigue5.png', 'fatigue6.png', 'fatigue7.png']


## Funciones Auxiliares (Carga y Guardado)

In [3]:
def cargar_imagen_gris(path):
    """Carga una imagen, la convierte a gris y la normaliza a float [0, 1]."""
    try:
        # Usamos as_gray=True para que imread maneje la conversión
        img_gray = imread(path, as_gray=True)
        # 'imread' con 'as_gray=True' ya devuelve float [0, 1]
        img_norm = img_as_float(img_gray)
        return img_norm
    except Exception as e:
        print(f"Error cargando la imagen {path}: {e}")
        return None

def save_fig(path): 
    try:
        full_path = RESULTS_DIR / path
        plt.savefig(full_path, bbox_inches='tight', dpi=150)
        print(f"Figura guardada en: {full_path}")
    except Exception as e:
        print(f"Error guardando figura: {e}")

## Funciones de Procesamiento de Imagen

In [4]:
def mostrar_histograma(ax, img_gray_norm, title='Histograma'):
    """Muestra el histograma de una imagen en un eje (ax) dado."""
    ax.hist(img_gray_norm.ravel(), bins=256, range=(0.0, 1.0), fc='k', ec='k')
    ax.set_title(title)
    ax.set_xlabel('Intensidad de Píxel (0-1)')
    ax.set_ylabel('Frecuencia')
    ax.set_xlim(0, 1)

def procesar_clahe(img_gray_norm, clip_limit=0.03):
    """Aplica CLAHE (Ecualización Adaptativa) a la imagen."""
    return exposure.equalize_adapthist(img_gray_norm, clip_limit=clip_limit)

def procesar_sobel(img_gray_norm):
    """Aplica el filtro Sobel para detección de bordes."""
    return filters.sobel(img_gray_norm)

def procesar_canny(img_gray_norm, sigma=1.0):
    """Aplica el detector de bordes Canny."""
    # Corregido: Se llama a 'canny' directamente, no a 'filters.canny'
    return canny(img_gray_norm, sigma=sigma)

def procesar_otsu(img_gray_norm):
    """Aplica umbralización global de Otsu."""
    thresh = filters.threshold_otsu(img_gray_norm)
    return img_gray_norm > thresh

def procesar_morfologia(img_bin, op_type='opening', disk_size=3):
    """Aplica operaciones morfológicas (opening, closing, erosion, dilation)."""
    selem = morphology.disk(disk_size)
    if op_type == 'opening':
        return morphology.binary_opening(img_bin, selem)
    elif op_type == 'closing':
        return morphology.binary_closing(img_bin, selem)
    elif op_type == 'erosion':
        return morphology.binary_erosion(img_bin, selem)
    elif op_type == 'dilation':
        return morphology.binary_dilation(img_bin, selem)
    return img_bin

def procesar_etiquetado(img_bin):
    """Etiqueta regiones conectadas y muestra sus propiedades."""
    label_image, num_features = ndi.label(img_bin)
    props = measure.regionprops_table(label_image, 
                                      properties=['label', 'area', 'perimeter', 'eccentricity', 'major_axis_length'])
    print(f"Número de objetos encontrados: {num_features}")
    # print("Propiedades de los objetos:")
    # print(props) # Descomentar para ver la tabla de propiedades
    return label_image, props

## Clase para Calibración y Medición

In [5]:
class CalculadoraDistancia:
    """
    Clase para manejar los clics en la figura de matplotlib para
    calibración y medición.
    """
    def __init__(self, fig, ax, img, out_calib, out_measure):
        self.fig = fig
        self.ax = ax
        self.img = img
        self.out_calib = out_calib
        self.out_measure = out_measure
        
        self.pix_per_um = None
        self._calib_points = []
        self._measure_points = []
        self.mode = 'idle' # Modos: 'idle', 'calibrating', 'measuring'
        
        self.ax.imshow(self.img, cmap='gray')
        self.ax.set_title("Haz clic en 'Calibrar' o 'Medir'")
        self.ax.axis('off')
        
        # Conectar el evento de clic
        self.cid = self.fig.canvas.mpl_connect('button_press_event', self._handle_click)

    def reset_image(self, img):
        """Carga una nueva imagen en la calculadora."""
        self.img = img
        self.ax.clear()
        self.ax.imshow(self.img, cmap='gray')
        self.ax.set_title("Imagen cargada. Lista para calibrar.")
        self.ax.axis('off')
        self.fig.canvas.draw()
        self.reset_calibration()
        
    def reset_calibration(self):
        """Resetea el estado de calibración y medición."""
        self.pix_per_um = None
        self._calib_points = []
        self._measure_points = []
        self.mode = 'idle'
        
        # Limpiar líneas previas del eje
        for line in self.ax.get_lines():
            line.remove()
        self.fig.canvas.draw()

    def start_calibration(self):
        """Inicia el modo de calibración."""
        self.reset_calibration()
        self.mode = 'calibrating'
        with self.out_calib:
            print("MODO CALIBRACIÓN: Haz clic en los dos extremos de la barra de escala.")
        self.ax.set_title("MODO CALIBRACIÓN: Clic en P1 de la escala")

    def set_scale(self, real_distance_um):
        """Calcula y guarda la escala (pixeles por micrón)."""
        if len(self._calib_points) != 2 or real_distance_um <= 0:
            with self.out_calib:
                print("Error: Se necesitan 2 puntos y una distancia > 0.")
            return

        p1, p2 = self._calib_points
        pixel_distance = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
        self.pix_per_um = pixel_distance / real_distance_um
        
        self.mode = 'idle'
        with self.out_calib:
            clear_output(wait=True)
            print(f"CALIBRACIÓN COMPLETA:")
            print(f"  Distancia en píxeles: {pixel_distance:.2f} px")
            print(f"  Distancia real: {real_distance_um} μm")
            print(f"  Escala: {self.pix_per_um:.2f} pix/μm")
        self.ax.set_title(f"Escala: {self.pix_per_um:.2f} pix/μm. Listo para medir.")

    def start_measuring(self):
        """Inicia el modo de medición."""
        if self.pix_per_um is None:
            with self.out_measure:
                print("Error: Debes calibrar la escala primero.")
            return
        
        self._measure_points = []
        self.mode = 'measuring'
        with self.out_measure:
            print("MODO MEDICIÓN: Haz clic en dos puntos para medir la distancia.")
        self.ax.set_title("MODO MEDICIÓN: Clic en P1")

    def _handle_click(self, event):
        """Manejador de eventos de clic del mouse."""
        if event.inaxes != self.ax:
            return
        
        x, y = event.xdata, event.ydata
        
        if self.mode == 'calibrating':
            self._calib_points.append((x, y))
            self.ax.plot(x, y, 'r+', markersize=10) # Marcar punto
            self.fig.canvas.draw()
            
            if len(self._calib_points) == 1:
                self.ax.set_title("MODO CALIBRACIÓN: Clic en P2 de la escala")
            elif len(self._calib_points) == 2:
                self.mode = 'idle'
                p1, p2 = self._calib_points
                self.ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'r-') # Dibujar línea
                self.fig.canvas.draw()
                self.ax.set_title("CALIBRACIÓN: Ingresa la distancia (μm) y presiona 'Fijar Escala'")
                with self.out_calib:
                    print("Puntos de calibración seleccionados. Ingresa la distancia real.")

        elif self.mode == 'measuring':
            self._measure_points.append((x, y))
            self.ax.plot(x, y, 'gx', markersize=10) # Marcar punto
            self.fig.canvas.draw()
            
            if len(self._measure_points) == 1:
                self.ax.set_title("MODO MEDICIÓN: Clic en P2")
            elif len(self._measure_points) == 2:
                self.mode = 'idle'
                p1, p2 = self._measure_points
                self.ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'g--') # Dibujar línea
                
                pixel_distance = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
                real_distance = pixel_distance / self.pix_per_um
                
                with self.out_measure:
                    clear_output(wait=True)
                    print("MEDICIÓN COMPLETA:")
                    print(f"  Distancia en píxeles: {pixel_distance:.2f} px")
                    print(f"  Distancia real: {real_distance:.2f} μm")
                
                self.ax.set_title(f"Medición: {real_distance:.2f} μm. Inicia otra medida.")
                # Reiniciar para la siguiente medida
                self.start_measuring()

## Interfaz Interactiva y Lógica Principal

In [6]:
# Celda 6 (Corregida para gráficas en blanco)

# --- Validar si se encontraron imágenes ---
if not image_options:
    print("No se pueden iniciar los widgets porque no se encontraron imágenes.")
    print(f"Asegúrate de colocar tus archivos en: {FILES_DIR.resolve()}")
else:
    # --- 1. Crear todos los widgets ---

    # Layouts para organizar
    layout_50_pct = Layout(width='50%')
    layout_btn = Layout(width='150px')
    
    # --- Panel de Selección ---
    image_dropdown = widgets.Dropdown(
        options=image_options,
        description='Imagen:',
        style={'description_width': 'initial'},
        layout=layout_50_pct
    )

    # --- Panel de Calibración ---
    btn_calibrate = widgets.Button(description='1. Calibrar', layout=layout_btn)
    text_um = widgets.FloatText(
        value=10.0, 
        description='Distancia (μm):', 
        style={'description_width': 'initial'},
        layout=Layout(width='200px')
    )
    btn_set_scale = widgets.Button(description='2. Fijar Escala', layout=layout_btn)
    
    # --- Panel de Medición ---
    btn_measure = widgets.Button(description='3. Medir Distancia', layout=layout_btn)

    # --- Áreas de Salida ---
    # Output para la figura interactiva de medición
    out_fig_interactive = Output()
    
    # Outputs para mensajes de texto
    out_calib_text = Output()
    out_measure_text = Output()
    
    # Output para todas las demás figuras de análisis
    out_analysis_plots = Output()

    # --- 2. Inicializar la Lógica y la Figura ---
    
    # Cargar la primera imagen por defecto
    default_img_path = image_path_map[image_options[0]]
    current_img_gray = cargar_imagen_gris(default_img_path)
    
    # --- VALIDACIÓN ---
    # Verificar si la primera imagen se cargó correctamente
    if current_img_gray is None:
        print(f"ERROR FATAL: No se pudo cargar la imagen por defecto: {default_img_path}")
        print(f"Verifica que el archivo exista en '{FILES_DIR.resolve()}', no esté corrupto y tengas permisos.")
        print("La interfaz interactiva no se iniciará.")
    else:
        # --- Fin VALIDACIÓN (todo el código restante va dentro de este 'else') ---

        # Crear la figura interactiva y la clase calculadora
        with out_fig_interactive:
            fig_interactive, ax_interactive = plt.subplots(figsize=(7, 6))
            plt.tight_layout()
            plt.show()

        dist_calculator = CalculadoraDistancia(
            fig_interactive, 
            ax_interactive, 
            current_img_gray, 
            out_calib_text, 
            out_measure_text
        )

        # --- 3. Definir Observadores (Handlers) ---

        def run_full_analysis(img_gray, base_filename):
            """Ejecuta todos los análisis y los muestra en out_analysis_plots."""
            with out_analysis_plots:
                clear_output(wait=True)
                print(f"--- Iniciando análisis para: {base_filename} ---")
                
                # Crear una figura para todos los análisis
                fig_analysis, axs = plt.subplots(3, 2, figsize=(10, 12))
                axs = axs.ravel() # Aplanar el array de ejes
                
                # a) Histograma (sobre la imagen float original)
                mostrar_histograma(axs[0], img_gray, 'Histograma Original')
                
                # b) CLAHE (creado como float)
                img_clahe = procesar_clahe(img_gray)
                axs[1].imshow(img_clahe, cmap='gray')
                axs[1].set_title('Ecualización (CLAHE)')
                axs[1].axis('off')
                save_fig(f"{base_filename}_01_clahe.png")

                # --- INICIO DE CORRECCIÓN ---
                # Convertir la imagen CLAHE a uint8 [0, 255]
                # Esta versión se usará para las funciones (sobel, canny, otsu)
                img_clahe_u8 = img_as_ubyte(img_clahe)
                # --- FIN DE CORRECCIÓN ---

                # c) Sobel (usando la versión uint8)
                img_sobel = procesar_sobel(img_clahe_u8) # Modificado
                axs[2].imshow(img_sobel, cmap='gray')
                axs[2].set_title('Filtro Sobel (sobre CLAHE)')
                axs[2].axis('off')
                save_fig(f"{base_filename}_02_sobel.png")

                # d) Canny (usando la versión uint8)
                img_canny = procesar_canny(img_clahe_u8, sigma=2.0) # Modificado
                axs[3].imshow(img_canny, cmap='binary')
                axs[3].set_title('Bordes Canny (sigma=2)')
                axs[3].axis('off')
                save_fig(f"{base_filename}_03_canny.png")

                # e) Otsu (usando la versión uint8)
                img_otsu = procesar_otsu(img_clahe_u8) # Modificado
                axs[4].imshow(img_otsu, cmap='gray')
                axs[4].set_title('Umbralización Otsu (sobre CLAHE)')
                axs[4].axis('off')
                save_fig(f"{base_filename}_04_otsu.png")

                # f) Morfología (Opening, sobre la imagen de Otsu que ya es booleana)
                img_opened = procesar_morfologia(img_otsu, op_type='opening', disk_size=2)
                axs[5].imshow(img_opened, cmap='gray')
                axs[5].set_title('Morfología (Opening, disk=2)')
                axs[5].axis('off')
                save_fig(f"{base_filename}_05_opening.png")
                
                plt.tight_layout()
                plt.show()
                
                # g) Etiquetado (se imprime en la consola)
                print("\n--- Análisis de Regiones (sobre imagen binarizada con Otsu+Opening) ---")
                procesar_etiquetado(img_opened)
                

        def on_image_select(change):
            """Se activa al cambiar la imagen en el dropdown."""
            if change['type'] == 'change' and change['name'] == 'value':
                selected_file = change['new']
                file_path = image_path_map[selected_file]
                
                # Limpiar todos los outputs
                out_calib_text.clear_output()
                out_measure_text.clear_output()
                out_analysis_plots.clear_output()
                
                # Cargar nueva imagen
                new_img_gray = cargar_imagen_gris(file_path)
                if new_img_gray is not None:
                    # Actualizar la figura interactiva
                    dist_calculator.reset_image(new_img_gray)
                    
                    # Ejecutar el análisis completo para la nueva imagen
                    base_filename = Path(selected_file).stem
                    run_full_analysis(new_img_gray, base_filename)
                else:
                    with out_analysis_plots:
                        clear_output(wait=True)
                        print(f"ERROR: No se pudo cargar la imagen seleccionada: {selected_file}")

        def on_calibrate_click(b):
            """Se activa al pulsar 'Calibrar'."""
            out_calib_text.clear_output()
            out_measure_text.clear_output()
            dist_calculator.start_calibration()

        def on_set_scale_click(b):
            """Se activa al pulsar 'Fijar Escala'."""
            real_dist = text_um.value
            dist_calculator.set_scale(real_dist)

        def on_measure_click(b):
            """Se activa al pulsar 'Medir Distancia'."""
            out_measure_text.clear_output()
            dist_calculator.start_measuring()

        # --- 4. Conectar Observadores ---
        image_dropdown.observe(on_image_select)
        btn_calibrate.on_click(on_calibrate_click)
        btn_set_scale.on_click(on_set_scale_click)
        btn_measure.on_click(on_measure_click)

        # --- 5. Mostrar la Interfaz ---
        
        # Organizar layout
        panel_seleccion = VBox([
            widgets.HTML("<b>Paso A: Seleccionar Imagen</b>"), 
            image_dropdown
        ])
        
        panel_calibracion = VBox([
            widgets.HTML("<b>Paso B: Calibración de Escala</b>"),
            HBox([btn_calibrate, text_um, btn_set_scale]),
            out_calib_text
        ])
        
        panel_medicion = VBox([
            widgets.HTML("<b>Paso C: Medición</b>"),
            btn_measure,
            out_measure_text
        ])
        
        panel_interactivo = VBox([
            widgets.HTML("<h3>Herramienta de Medición Interactiva</h3>"),
            out_fig_interactive,
            panel_calibracion,
            panel_medicion
        ])
        
        panel_analisis = VBox([
            widgets.HTML("<h3>Análisis de Imagen (Resultados)</h3>"),
            out_analysis_plots
        ])

        # Lanzar el análisis para la primera imagen
        on_image_select({'type': 'change', 'name': 'value', 'new': image_options[0]})

        # Mostrar todo
        display(panel_seleccion, HBox([panel_interactivo, panel_analisis]))